<a href="https://colab.research.google.com/github/amanotk/numerical-geophysics/blob/main/notebook/EulerEquation.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab">
</a>

# Euler方程式

Euler方程式（圧縮性流体方程式）の数値解を求める．
$$
\frac{\partial}{\partial t}
\begin{pmatrix}
\rho \\ \rho v \\ \varepsilon
\end{pmatrix} +
\frac{\partial}{\partial x}
\begin{pmatrix}
\rho v \\ \rho v^2 + p \\ (\varepsilon + p) v
\end{pmatrix} = 0
$$
ここで
$$
\varepsilon = \frac{1}{2} \rho v^2 + \frac{p}{\gamma-1}
$$
は流体の全エネルギー密度である．

In [ ]:
# 準備
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt

In [ ]:
# フォントサイズ
plt.rcParams["font.size"] = 14

In [ ]:
def push_lw2(uu, gamma, dt, dx, step, epsilon):
  "2段階Lax-Wendroffスキーム"
  dtx = dt/dx
  Nx = np.shape(uu)[0] - 2
  ix = np.arange(1, Nx+1, dtype=np.int32)
  hx = np.arange(0, Nx+1, dtype=np.int32) # i+1/2
  uh = np.zeros_like(uu)
  fx = np.zeros_like(uu)
  for n in range(step):
    # 1段階目
    ro = uu[:,0]
    vx = uu[:,1]/(ro + 1.0e-20)
    pr = (gamma - 1)*(uu[:,2] - 0.5*ro*vx**2)
    fx[:,0] = ro*vx
    fx[:,1] = ro*vx*vx + pr
    fx[:,2] = (0.5*ro*vx*vx + gamma/(gamma-1)*pr)*vx
    uh[hx,:] = 0.5*((uu[hx+1,:] + uu[hx,:]) - dtx*(fx[hx+1,:] - fx[hx,:]))
    # 人工粘性の係数を決定
    kappa = 1/dtx*epsilon*np.abs(vx[hx+1] - vx[hx])[:,None]
    # 2段階目
    ro = uh[:,0]
    vx = uh[:,1]/(ro + 1.0e-20)
    pr = (gamma - 1)*(uh[:,2] - 0.5*ro*vx**2)
    fx[:,0] = ro*vx
    fx[:,1] = ro*vx*vx + pr
    fx[:,2] = (0.5*ro*vx*vx + gamma/(gamma-1)*pr)*vx
    fx[hx,:] = fx[hx,:] - kappa*(uu[hx+1,:] - uu[hx,:])
    uu[ix,:] = uu[ix,:] - dtx*(fx[ix,:] - fx[ix-1,:])
    # 境界条件
    uu[   0,:] = uu[Nx,:]
    uu[Nx+1,:] = uu[ 1,:]

## 音波の伝播

エントロピー$S = p/\rho^{\gamma}$が一定のとき
$$
\frac{\partial}{\partial t} J_{\pm} + (v \pm C_s)
\frac{\partial}{\partial x} J_{\pm} = 0
\quad
\left( J_{\pm} = v \pm \frac{2 C_s}{\gamma-1} \right)
$$
が成り立つ．
したがって，エントロピーおよび$J_{-}$が一定の初期条件を選ぶことで，正方向に伝播する音波のみを考えることができる．

ここでは計算領域を$0 \leq x \leq 1$とし，初期条件
$$
\begin{align}
&\rho = \rho_0 + \rho_1 \sin \left( 2 \pi x \right)
\\
&v = \frac{2 C_{s,0}}{\gamma-1}
\left[
  \left( 1 + \frac{\rho_1}{\rho_0} \sin (2 \pi x) \right)^{(\gamma-1)/2}
  - 1
\right]
\\
&p = p_0
\left(
  1 + \frac{\rho_1}{\rho_0} \sin (2 \pi x)
\right)^{\gamma}
\end{align}
$$
を考えよう．ただし$\rho_0 = 1$, $p_0 = 1/\gamma$, $C_{s,0} = (\gamma p_0/\rho_0)^{1/2}$, $\gamma = 5/3$とする．  
このとき$S = const$，$J_{-} = const$となることは簡単に確かめられる．  

音波の振幅$\rho_1$が十分小さければ，ゆらぎは線形の音波として伝播するが，有限振幅では急峻化によって衝撃波が形成する．（衝撃波が形成すると散逸によってエントロピーが生成されるので$J_{-}$も一定ではなくなる．）

In [ ]:
def set_initial(Nx, gamma, delta):
  dx = 1.0/Nx
  xx = dx*np.arange(Nx+2) - 0.5*dx
  uu = np.zeros((Nx+2, 3))
  ro = 1 + delta * np.sin(2*np.pi*xx)
  vx = 2/(gamma-1) * (ro**((gamma-1)/2) - 1)
  pr = ro**gamma / gamma
  uu[:,0] = ro
  uu[:,1] = ro*vx
  uu[:,2] = 0.5*ro*vx**2 + pr/(gamma-1)
  return xx, dx, uu

def push_lw2(uu, gamma, dt, dx, step, epsilon):
  "2段階Lax-Wendroffスキーム"
  dtx = dt/dx
  Nx = np.shape(uu)[0] - 2
  ix = np.arange(1, Nx+1, dtype=np.int32)
  hx = np.arange(0, Nx+1, dtype=np.int32) # i+1/2
  uh = np.zeros_like(uu)
  fx = np.zeros_like(uu)
  for n in range(step):
    # 1段階目
    ro = uu[:,0]
    vx = uu[:,1]/(ro + 1.0e-20)
    pr = (gamma - 1)*(uu[:,2] - 0.5*ro*vx**2)
    fx[:,0] = ro*vx
    fx[:,1] = ro*vx*vx + pr
    fx[:,2] = (0.5*ro*vx*vx + gamma/(gamma-1)*pr)*vx
    uh[hx,:] = 0.5*((uu[hx+1,:] + uu[hx,:]) - dtx*(fx[hx+1,:] - fx[hx,:]))
    # 人工粘性の係数を決定
    kappa = 1/dtx*epsilon*np.abs(vx[hx+1] - vx[hx])[:,None]
    # 2段階目
    ro = uh[:,0]
    vx = uh[:,1]/(ro + 1.0e-20)
    pr = (gamma - 1)*(uh[:,2] - 0.5*ro*vx**2)
    fx[:,0] = ro*vx
    fx[:,1] = ro*vx*vx + pr
    fx[:,2] = (0.5*ro*vx*vx + gamma/(gamma-1)*pr)*vx
    fx[hx,:] = fx[hx,:] - kappa*(uu[hx+1,:] - uu[hx,:])
    uu[ix,:] = uu[ix,:] - dtx*(fx[ix,:] - fx[ix-1,:])
    # 境界条件
    uu[   0,:] = uu[Nx,:]
    uu[Nx+1,:] = uu[ 1,:]

# 振幅 delta = rho_1/rho_0
delta = 5.0e-2

# パラメータ
gamma   = 5.0/3.0
epsilon = 1.0e-1
Nx      = 100
nu      = 0.25
ix      = np.arange(1, Nx+1)

# 初期条件
xx, dx, uu = set_initial(Nx, gamma, delta)
dt = nu*dx

# プロット
fig, axs = plt.subplots(4, 1, figsize=(10, 10))

step = 40
for n in range(5):
  ro = uu[:,0]
  vx = uu[:,1]/(ro + 1.0e-20)
  pr = (gamma - 1)*(uu[:,2] - 0.5*ro*vx**2)
  Jp = vx + np.sqrt(gamma*pr/ro) * 2/(gamma-1)
  Jm = vx - np.sqrt(gamma*pr/ro) * 2/(gamma-1)
  axs[0].plot(xx[ix], ro[ix], label='t = {:4.2f}'.format(n*step*dt))
  axs[1].plot(xx[ix], vx[ix], label='t = {:4.2f}'.format(n*step*dt))
  axs[2].plot(xx[ix], Jp[ix], label='t = {:4.2f}'.format(n*step*dt))
  axs[3].plot(xx[ix], Jm[ix], label='t = {:4.2f}'.format(n*step*dt))
  push_lw2(uu, gamma, dt, dx, step, epsilon)

axs[0].set_title(r'$\rho_1/\rho_0 = {:8.1e}$'.format(delta))
axs[0].set_ylabel(r'$\rho$')
axs[0].set_ylim(1 - 1.5*delta, 1 + 1.5*delta)
axs[1].set_ylabel(r'$v$')
axs[1].set_ylim(-1.5*delta, 1.5*delta)
axs[2].set_ylabel(r'$J_{+}$')
axs[2].set_ylim(+3*(1 + delta), +3*(1 - delta))
axs[3].set_ylabel(r'$J_{-}$')
axs[3].set_ylim(-3*(1 + delta), -3*(1 - delta))
axs[3].set_xlabel('x')
axs[0].legend(loc='upper left', bbox_to_anchor=(1.01, 1.0))

for ax in axs:
  ax.set_xlim(0.0, 1.0)

## Riemann問題（衝撃波管問題）

初期条件として
$$
(\rho, v, p)
=
\begin{cases}
(\rho_L, v_L, p_L) \quad x < 0
\\
(\rho_R, v_R, p_R) \quad x > 0
\end{cases}
$$
のように，$x = 0$で異なる流体が接しているときの時間発展を考える．

ここでは
$$
(\rho_L, v_L, p_L) = (1, 0, 1), \quad
(\rho_R, v_R, p_R) = (0.125, 0, 0.1)
$$
かつ$\gamma = 1.4$の場合を考えよう．  
これは"Sod's Problem"として知られる衝撃波管の最も一般的なテスト問題である．

計算領域は$-1 \leq x \leq 1$，境界条件はどちらも$\partial/\partial x = 0$とする．

In [ ]:
def set_initial(Nx, gamma, delta):
  dx = 2.0/Nx
  xx = dx*np.arange(Nx+2) - (1 + 0.5*dx)
  uu = np.zeros((Nx+2, 3))
  ro = np.where(xx < 0, 1.0, 0.125)
  vx = np.where(xx < 0, 0.0, 0.0)
  pr = np.where(xx < 0, 1.0, 0.1)
  uu[:,0] = ro
  uu[:,1] = ro*vx
  uu[:,2] = 0.5*ro*vx**2 + pr/(gamma-1)
  return xx, dx, uu

def push_lw2(uu, gamma, dt, dx, step, epsilon):
  "2段階Lax-Wendroffスキーム"
  dtx = dt/dx
  Nx = np.shape(uu)[0] - 2
  ix = np.arange(1, Nx+1, dtype=np.int32)
  hx = np.arange(0, Nx+1, dtype=np.int32) # i+1/2
  uh = np.zeros_like(uu)
  fx = np.zeros_like(uu)
  for n in range(step):
    # 1段階目
    ro = uu[:,0]
    vx = uu[:,1]/(ro + 1.0e-20)
    pr = (gamma - 1)*(uu[:,2] - 0.5*ro*vx**2)
    fx[:,0] = ro*vx
    fx[:,1] = ro*vx*vx + pr
    fx[:,2] = (0.5*ro*vx*vx + gamma/(gamma-1)*pr)*vx
    uh[hx,:] = 0.5*((uu[hx+1,:] + uu[hx,:]) - dtx*(fx[hx+1,:] - fx[hx,:]))
    # 人工粘性の係数を決定
    kappa = 1/dtx*epsilon*np.abs(vx[hx+1] - vx[hx])[:,None]
    # 2段階目
    ro = uh[:,0]
    vx = uh[:,1]/(ro + 1.0e-20)
    pr = (gamma - 1)*(uh[:,2] - 0.5*ro*vx**2)
    fx[:,0] = ro*vx
    fx[:,1] = ro*vx*vx + pr
    fx[:,2] = (0.5*ro*vx*vx + gamma/(gamma-1)*pr)*vx
    fx[hx,:] = fx[hx,:] - kappa*(uu[hx+1,:] - uu[hx,:])
    uu[ix,:] = uu[ix,:] - dtx*(fx[ix,:] - fx[ix-1,:])
    # 境界条件
    uu[   0,:] = uu[ 1,:]
    uu[Nx+1,:] = uu[Nx,:]



# パラメータ
gamma   = 1.4
epsilon = 5.0e-1
Nx      = 200
nu      = 0.25
ix      = np.arange(1, Nx+1)

# 初期条件
xx, dx, uu = set_initial(Nx, gamma, delta)
dt = nu*dx

# プロット
fig, axs = plt.subplots(1, 1, figsize=(8, 6))

# 初期条件
ro = uu[:,0]
vx = uu[:,1]/(ro + 1.0e-20)
pr = (gamma - 1)*(uu[:,2] - 0.5*ro*vx**2)
plt.plot(xx[ix], ro[ix], 'k--')
plt.plot(xx[ix], vx[ix], 'r--')
plt.plot(xx[ix], pr[ix], 'b--')

step = int(0.40 / dt)
push_lw2(uu, gamma, dt, dx, step, epsilon)
ro = uu[:,0]
vx = uu[:,1]/(ro + 1.0e-20)
pr = (gamma - 1)*(uu[:,2] - 0.5*ro*vx**2)
plt.plot(xx[ix], ro[ix], 'k-', label='density')
plt.plot(xx[ix], vx[ix], 'r-', label='velocity')
plt.plot(xx[ix], pr[ix], 'b-', label='pressure')

plt.xlim(-1.0, 1.0)
plt.title(r'$t = {:4.2f}$'.format(step*dt))
plt.legend(loc='upper left', bbox_to_anchor=(1.0, 1.0))